<a href="https://colab.research.google.com/github/rajgupt/my-colab-notebooks/blob/master/orientation_correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [0]:
os.chdir('/content/drive/My Drive/colab/data')

In [0]:
!ls

Data source: https://guillaumejaume.github.io/FUNSD/dataset.zip

# Dataset Preparation

In [0]:
!mkdir -p funsd_dataset/rotate/0

In [0]:
!cp funsd_dataset/training_data/images/* funsd_dataset/rotate/0/

In [0]:
!cp funsd_dataset/testing_data/images/* funsd_dataset/rotate/0/

In [0]:
from PIL import Image
import glob
# if not os.path.exists('funsd_dataset/rotate'):
images = glob.glob('funsd_dataset/rotate/0/*.png')
os.makedirs('funsd_dataset/rotate/90', exist_ok=True)
os.makedirs('funsd_dataset/rotate/180', exist_ok=True)
os.makedirs('funsd_dataset/rotate/270', exist_ok=True)
for angle in [90,180,270]:
  for im in images:
    img = Image.open(im)
    imname = os.path.basename(im)
    save_path = os.path.join('funsd_dataset/rotate/'+str(angle)+'/'+imname)
    img.rotate(angle).save(save_path)

# Orientation Prediction Model

Basic Classifier to predict the angle of orientation

In [0]:
def get_model():
  inputs = 

In [0]:
tf.__version__

In [0]:
import pathlib
datadir = pathlib.Path('funsd_dataset/rotate')

In [0]:
image_count = len(list(datadir.glob('*/*.png')))
image_count

In [0]:
CLASS_NAMES = np.array([item.name for item in datadir.glob('*')])
CLASS_NAMES